
# Assignment IV: Advanced CUDA
## [DD2360] Applied GPU Programming
Group 8

Valeria Grotto (vgrotto@kth.se) | Martin Forslund (uz6@kth.se)

***

## EX 2 - CUDA Streams


In [1]:
! git clone https://github.com/MoggmentuM/DD2360.git

Cloning into 'DD2360'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 138 (delta 36), reused 31 (delta 4), pack-reused 0
Receiving objects: 100% (138/138), 486.73 KiB | 4.59 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [12]:
# compile file
! nvcc /content/DD2360/hw_4/ex_2/vectorAdd.cu -o /content/DD2360/hw_4/ex_2/vectorAdd

In [16]:
! ./DD2360/hw_4/ex_2/vectorAdd 90000000

The input length is 90000000
vecAdd<<<(2812500,1), (32,1)>>> elapsed 0.336772 sec

---------------------------------------------
SUCCESS


### Profiling without streams

In [15]:
# to output file
! nvprof --output-profile /content/DD2360/hw_4/ex_2/output_trace.nvvp -f ./DD2360/hw_4/ex_2/vectorAdd 90000000

The input length is 90000000
==3826== NVPROF is profiling process 3826, command: ./DD2360/hw_4/ex_2/vectorAdd 90000000
vecAdd<<<(2812500,1), (32,1)>>> elapsed 0.320135 sec

---------------------------------------------
SUCCESS
==3826== Generated result file: /content/DD2360/hw_4/ex_2/output_trace.nvvp


In [10]:
! nvprof --print-gpu-trace ./DD2360/hw_4/ex_2/vectorAdd 90000000

The input length is 500000
==3031== NVPROF is profiling process 3031, command: ./DD2360/hw_4/ex_2/vectorAdd 500000
vecAdd<<<(15625,1), (32,1)>>> elapsed 0.000000 sec
==3031== Error: Internal profiling error 3959:1.

---------------------------------------------
SUCCESS
======== Profiling result:
   Start  Duration            Grid Size      Block Size     Regs*    SSMem*    DSMem*      Size  Throughput  SrcMemType  DstMemType           Device   Context    Stream  Name
     0ns  852.38us                    -               -         -         -         -  3.8147MB  4.3705GB/s    Pageable      Device     Tesla T4 (0)         1         7  [CUDA memcpy HtoD]
1.1264ms  759.35us                    -               -         -         -         -  3.8147MB  4.9059GB/s    Pageable      Device     Tesla T4 (0)         1         7  [CUDA memcpy HtoD]
1.8988ms  60.992us          (15625 1 1)        (32 1 1)        16        0B        0B         -           -           -           -     Tesla T4 (0)  

**Nvidia Nsight**

In [14]:
! /usr/local/cuda-11/bin/nv-nsight-cu-cli ./DD2360/hw_4/ex_2/vectorAdd 90000000

The input length is 90000000
==PROF== Connected to process 3672 (/content/DD2360/hw_4/ex_2/vectorAdd)
==PROF== Profiling "vecAdd" - 0: 0%....50%....100% - 8 passes
vecAdd<<<(2812500,1), (32,1)>>> elapsed 0.983644 sec

---------------------------------------------
SUCCESS
==PROF== Disconnected from process 3672
[3672] vectorAdd@127.0.0.1
  vecAdd(double *, double *, double *, int), 2023-Dec-10 12:57:32, Context 1, Stream 7
    Section: GPU Speed Of Light Throughput
    ---------------------------------------------------------------------- --------------- ------------------------------
    DRAM Frequency                                                           cycle/nsecond                           5.00
    SM Frequency                                                             cycle/usecond                         585.20
    Elapsed Cycles                                                                   cycle                      5,618,207
    Memory [%]                             

In [38]:
# compile file
! nvcc /content/DD2360/hw_4/ex_2/vectorAddStream.cu -o /content/DD2360/hw_4/ex_2/vectorAddStream

In [37]:
#run file
! ./DD2360/hw_4/ex_2/vectorAddStream 90000000

The input length is 90000000
Segment Size: 64, Time: 0.031416 sec
Segment Size: 128, Time: 0.031457 sec
Segment Size: 256, Time: 0.031412 sec
Segment Size: 512, Time: 0.031416 sec

---------------------------------------------
SUCCESS


In [39]:
# to output file
! nvprof --output-profile /content/DD2360/hw_4/ex_2/nvvp_traces/stream_trace.nvvp -f ./DD2360/hw_4/ex_2/vectorAddStream 90000000

The input length is 90000000
==5785== NVPROF is profiling process 5785, command: ./DD2360/hw_4/ex_2/vectorAddStream 90000000
Segment Size: 64, Time: 0.030009 sec
Segment Size: 128, Time: 0.029945 sec
Segment Size: 256, Time: 0.029678 sec
Segment Size: 512, Time: 0.028905 sec

---------------------------------------------
SUCCESS
==5785== Generated result file: /content/DD2360/hw_4/ex_2/nvvp_traces/stream_trace.nvvp


In [20]:
! /usr/local/cuda-11/bin/nv-nsight-cu-cli ./DD2360/hw_4/ex_2/vectorAddStream 90000000

The input length is 90000000
==PROF== Connected to process 4250 (/content/DD2360/hw_4/ex_2/vectorAddStream)
==PROF== Profiling "vecAdd" - 0: 0%....50%....100% - 8 passes
==PROF== Profiling "vecAdd" - 1: 0%....50%....100% - 8 passes
==PROF== Profiling "vecAdd" - 2: 0%....50%....100% - 8 passes
==PROF== Profiling "vecAdd" - 3: 0%....50%....100% - 8 passes
Segment Size: 64, Time: 0.929026 sec
==PROF== Profiling "vecAdd" - 4: 0%....50%....100% - 8 passes
==PROF== Profiling "vecAdd" - 5: 0%....50%....100% - 8 passes
==PROF== Profiling "vecAdd" - 6: 0%....50%....100% - 8 passes
==PROF== Profiling "vecAdd" - 7: 0%....50%....100% - 8 passes
Segment Size: 128, Time: 0.763867 sec
==PROF== Profiling "vecAdd" - 8: 0%....50%....100% - 8 passes
==PROF== Profiling "vecAdd" - 9: 0%....50%....100% - 8 passes
==PROF== Profiling "vecAdd" - 10: 0%....50%....100% - 8 passes
==PROF== Profiling "vecAdd" - 11: 0%....50%....100% - 8 passes
Segment Size: 256, Time: 0.771232 sec
==PROF== Profiling "vecAdd" - 12: